# Persistence and Efficency
You now might be aware of some of the problems with parsing several pages and how that can quicly get out of hand. While parsing, you might chose to persist the collected data, so that it can be analyzed and cleaned later. 
Parsing, retrieving, saving, and cleaning data are all separate actions, and you shouldn't try to work with data while collecting. In this notebook you'll practice further parsing techniques along with persistency, both for JSON and CSV formats as well as using SQL and a database.

Start by loading one of the available HTML files into the `scrapy` library

In [98]:
import scrapy
import os
current_dir = os.path.abspath('')
url = os.path.join(current_dir, "html/1992_World_Junior_Championships_in_Athletics_–_Men's_high_jump")
with open(url) as _f:
    url_data = _f.read()

response = scrapy.http.TextResponse(url, body=url_data, encoding='utf-8')

In [99]:
# Make sure that the interesting data is available. Interesting data means we have finalists and all the other athletes.
def find_finalists_table(response):
    for table in response.xpath('//table'):
        body = table.xpath('tbody')
        # check if len of body is more than 12, otherwise skip it
        if len(body.xpath('tr')) < 12:
            continue
        # now check if we have medalists
        for tr in body.xpath('tr'):
            if tr.xpath('td/span/img/@alt').extract_first():
                return table


1st Jagan Hames 2.23
2nd Antoine Burke 2.20
3rd Mika Polku 2.20
4 Attila Zsivoczky 2.20
5 Roland Stark 2.15
6 Oskari Frösén 2.15
7 Shunichi Kobayashi 2.10
7 Stefan Holm 2.10
9 Tomohiro Nomura 2.10
10 Henry Patterson 2.10
11 Bjørn Olsson 2.10
12 Stuart Ohrland 2.05


In [109]:
def process(response):
    table = find_finalists_table(response)
    results = []
    # extract the year from the title
    year = response.xpath('//title/text()').extract_first().split()[0]

    for tr in table.xpath('tbody').xpath('tr'):
        # extract the text value of alt attribute of the img tag
        medal = tr.xpath('td/span/img/@alt').extract_first()
        if medal and "medalist" in medal:
            place = medal.split()[0]
        else:
            try:
                place, _ = tr.xpath('td/text()').extract()
            except ValueError:
                pass
        try:
            athlete = tr.xpath('td/a/text()').extract_first()
        except ValueError:
            pass
    
        if not athlete and not medal:
            continue
        height = tr.xpath('td/b/text()').extract_first()
        #print(place, athlete, height, year)
        results.append([place, athlete, height, year])
    return results
        

In [113]:
parsed_data = []
for file in os.listdir(os.path.join(current_dir, "html")):
    url = os.path.join(current_dir, "html", file)
    with open(url) as _f:
        url_data = _f.read()
    response = scrapy.http.TextResponse(url, body=url_data, encoding='utf-8')
    parsed_data.append(process(response))

parsed_data

[[['1st', 'Alfredo Deza', '2.21', '1998'],
  ['2nd', 'Yin Xueli', '2.21', '1998'],
  ['3rd', 'Aleksandr Veryutin', '2.21', '1998'],
  ['4', 'Mike Ponikvar', '2.21', '1998'],
  ['5', 'Yaroslav Rybakov', '2.18', '1998'],
  ['6', 'Pawel Mankiewicz', '2.15', '1998'],
  ['7', 'Naoyuki Daigo', '2.15', '1998'],
  ['8', 'Ronald Garlett', '2.10', '1998'],
  ['8', 'Martin Lloyd', '2.10', '1998'],
  ['8', 'Casper Labuschagne', '2.10', '1998'],
  ['8', 'Nikolay Korostelyov', '2.10', '1998'],
  ['12', 'Einar Karl Hjartarson', '2.10', '1998'],
  ['12', 'Vusumzi Mtshatseni', '2.10', '1998'],
  ['14', 'James Carr', '2.10', '1998']],
 [['1st', 'Mark Boswell', '2.24', '1996'],
  ['2nd', 'Svatoslav Ton', '2.21', '1996'],
  ['2nd', 'Ben Challenger', '2.21', '1996'],
  ['4', 'Dave Furman', '2.18', '1996'],
  ['5', 'Toni Huikuri', '2.18', '1996'],
  ['6', 'Dejan Vreljakovic', '2.18', '1996'],
  ['7', 'James Brierley', '2.15', '1996'],
  ['7', 'Roman Fricke', '2.15', '1996'],
  ['9', 'François Potgieter', '2

This interaction with `scrapy` in a Jupyter Notebook is useful because you don't need to run the special shell and you also don't need to run the whole spider. Once you learn what you need to do here, you can adapat the spider to persist data.
First, start by persisting data as JSON. To do this, you will need to keep the information in a Python data structure like a dictionary, and then load it as a JSON object, and finally, save it to a file.

In [114]:
import json

# You can convert Python into JSON first, but there is no need if you use `json.dump()`
# as shown next
json_data = json.dumps(parsed_data)

# Persist it in a file:
with open("results.json", "w") as _f:
    # use dump() with the Python dictionary directly. 
    # the conversion is done on the fly
    json.dump(parsed_data, _f)

Now that you can persist the scrapped data as JSON, you can also use CSV. This is specially useful if you want to to some data science operations. Although you can use an advanced library like Pandas for this, you can use the standard library CSV module from Python.

In [ ]:
# construct the data first

column_names = ["Place", "Athlete", "Height", "Year"]
rows = []

for place, athlete, height, year in parsed_data:
    rows.append([place, athlete, height, year])


In [ ]:
# Now persist it to disk
import csv

with open("results.csv", "w") as _f:
    writer = csv.writer(_f)

    # write the column names
    writer.writerow(column_names)

    # now write the rows
    writer.writerows(rows)

Finally, you can persist data to a database. Unlike the JSON and CSV approach, using a database is much more memory efficient. This is the principal reason why you want to use a database instead of a file on disk. Imagine capturing 10GB of data. This could potentially mean that you need 10GB of available memory to hold onto that data before saving it to disk.
By using a database, you can save the data as the data is gathered. 

For the next cells, use a SQLite database to persist the data. Create the file-based database and the table needed.

In [ ]:
import sqlite3
connection = sqlite3.connect("1992_results.db")
db_table = 'CREATE TABLE results (id integer primary key, medal TEXT, athlete TEXT)'
cursor = connection.cursor()
cursor.execute(db_table)
connection.commit()

In [ ]:
# Now it is time to persist the data. Open the connection again
connection = sqlite3.connect("1992_results.db")
cursor = connection.cursor()
query = 'INSERT INTO results(medal, athlete) VALUES(?, ?)'

for tr in table.xpath('tr'):
    medal = tr.xpath('td/b/text()').extract()[0]
    athlete = tr.xpath('td/a/text()').extract()[0]
    cursor.execute(query, (medal, athlete)) 
    connection.commit()

The data is now persisted in a file-based database that you can query. Verify that all works by creating a new connection and querying the database.

Update the _wikipedia_ project and spider to use some of these techniques to persist data. Next, try parsing all the files in the _html_ directory instead of just one and persist all results. Do you think you can parse other information as well? 

Try parsing the height and the results from all the other athletes, not just the top three places.